# Combine data from TESS, Petigura, and Narita
Assuring same time offsets and ephem

In [42]:
import numpy as np
import matplotlib.pyplot as plt
import lightkurve as lk
from lightkurve import search_targetpixelfile
import pandas as pd
import os
import batman
from scipy.stats import norm
import matplotlib

In [43]:
### Define data directory
working_dir = os.getcwd()
data_dir = os.path.join(working_dir, "data")
print(f"Data directory: {data_dir}")

Data directory: /Users/pentrican10/Projects/k2-19-project/data


In [44]:
TESS_offset = 2457000
petigura_offset = 2454833

In [45]:
### read table from Petigura et al. 2020
file = "ajab5220t1_mrt.txt"
def read_table(file_name):
    ### path to table - Petigura et al 2020
    file_path = os.path.join(data_dir, file_name)

    ### Define the column names 
    columns = ["Planet", "Transit", "Inst", "Tc", "e_Tc", "Source"]

    ### Read the text file, specifying space as the delimiter, skipping model_guess_omc rows
    df = pd.read_csv(file_path, delim_whitespace=True, skiprows=22, names=columns)

    ### Remove NaN values
    df = df.dropna()
    return df

df_petigura = read_table(file)

/var/folders/69/wlx91x8s1x34mphlfjy0h4pm0000gn/T/ipykernel_69822/2146133021.py:11: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(file_path, delim_whitespace=True, skiprows=22, names=columns)


In [46]:
### Petigura data
planet_b_data = df_petigura[df_petigura["Planet"] == "K2-19b"]
planet_c_data = df_petigura[df_petigura["Planet"] == "K2-19c"]

petigura_ind = np.array(planet_b_data["Transit"]) 

period_b_petigura = 7.9222 # [days]
tc_b_petigura = 2027.9023 # [days] using petigura offset
period_c_petigura = 11.8993 # [days]
tc_c_petigura = 2020.0007 # [days] using petigura offset

### transit times
tc_petigura_b = np.array(planet_b_data["Tc"])
tc_petigura_c = np.array(planet_c_data["Tc"])

### update index (misalignment in paper indices and tc)
petigura_ind_b_updated = np.array(planet_b_data["Transit"]) - 6
petigura_ind_c_updated = np.array(planet_c_data["Transit"]) - 3

In [47]:
### Narita data
### get Narita et al times (K2)
df_narita = pd.read_csv('narita_times.txt', delim_whitespace=True)
df_narita_b = df_narita[df_narita["planet_num"] == 1]
tc_narita_b = np.array(df_narita_b["Tc"]) - petigura_offset
df_narita_c = df_narita[df_narita["planet_num"] == 2]
tc_narita_c = np.array(df_narita_c["Tc"]) - petigura_offset

### put times into petigura ephem (shift epoch by -6)
narita_ind_b = [-6, -5, -4, -3, -2, -1, 0, 1, 2, 3]
narita_ind_b = np.array(narita_ind_b)
narita_ind_c = [-3, -2, -1, 0, 1, 2, 3]
narita_ind_c = np.array(narita_ind_c)

/var/folders/69/wlx91x8s1x34mphlfjy0h4pm0000gn/T/ipykernel_69822/4284132366.py:3: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df_narita = pd.read_csv('narita_times.txt', delim_whitespace=True)


In [48]:
### TESS data
### tess times in the paper ephem
tnum_tess = [337,339,340,341,342,343,430,432]

### updated indices for petigura ephem
df_tess = pd.read_csv('tess_transit_data.csv')
tess_ind_b = np.array(df_tess["Transit"]) + 337
tc_tess_b = np.array(df_tess["Tc(TESS)"])

### Planet b

In [49]:
### collect all data together
### collect all transit indices
all_transit_num_b = []
for i in range(len(narita_ind_b)):
    all_transit_num_b.append(narita_ind_b[i])
for i in range(len(petigura_ind_b_updated)):
    all_transit_num_b.append(petigura_ind_b_updated[i])
for i in range(len(tess_ind_b)):
    all_transit_num_b.append(tess_ind_b[i])
# print(f"All transit indices: {all_transit_num_b}")

### collect all observed transit times
all_obs_tc_b = []
for i in range(len(tc_narita_b)):
    all_obs_tc_b.append(tc_narita_b[i])
for i in range(len(tc_petigura_b)):
    all_obs_tc_b.append(tc_petigura_b[i])
for i in range(len(tc_tess_b)):
    all_obs_tc_b.append(tc_tess_b[i])
# print(f"All observed transit times: {all_obs_tc_b}")

### collect all observed transit time errors
all_obs_tc_b_err = []
tc_b_err_narita = np.array(df_narita_b["Tc_err"])
tc_b_err_tess = np.array(df_tess["Tc_err"])
tc_b_err_petigura = np.array(planet_b_data["e_Tc"])
for i in range(len(tc_b_err_narita)):
    all_obs_tc_b_err.append(tc_b_err_narita[i])
for i in range(len(tc_b_err_petigura)):
    all_obs_tc_b_err.append(tc_b_err_petigura[i])
for i in range(len(tc_b_err_tess)):
    all_obs_tc_b_err.append(tc_b_err_tess[i])
# print(f"All observed transit times error: {all_obs_tc_b_err}")

### collect all calculated transit times with petigura ephem
all_calc_tc_b = []
for i in range(len(all_transit_num_b)):
    ind = tc_b_petigura + (all_transit_num_b[i] * period_b_petigura)
    all_calc_tc_b.append(ind)
# print(f"All calculated transit times: {all_calc_tc_b}")

### collect all omc
all_omc_b = []
for i in range(len(all_calc_tc_b)):
    omc_ = all_obs_tc_b[i] - all_calc_tc_b[i]
    all_omc_b.append(omc_)
# print(f"All OMC: {all_omc_b}")

all_omc_b_err = all_obs_tc_b_err
# print(f"All OMC err: {all_omc_b_err}")

### Planet c

In [50]:
### collect all values for planet c
### collect all transit indices
all_transit_num_c = []
for i in range(len(narita_ind_c)):
    all_transit_num_c.append(narita_ind_c[i])
for i in range(len(petigura_ind_c_updated)):
    all_transit_num_c.append(petigura_ind_c_updated[i])

# print(f"All transit indices c: {all_transit_num_c}")

### collect all observed transit times
all_obs_tc_c = []
for i in range(len(tc_narita_c)):
    all_obs_tc_c.append(tc_narita_c[i])
for i in range(len(tc_petigura_c)):
    all_obs_tc_c.append(tc_petigura_c[i])

# print(f"All observed transit times c: {all_obs_tc_c}")

### collect all observed transit time errors
all_obs_tc_c_err = []
tc_c_err_narita = np.array(df_narita_c["Tc_err"])
tc_c_err_petigura = np.array(planet_c_data["e_Tc"])
for i in range(len(tc_c_err_narita)):
    all_obs_tc_c_err.append(tc_c_err_narita[i])
for i in range(len(tc_c_err_petigura)):
    all_obs_tc_c_err.append(tc_c_err_petigura[i])
# print(f"All observed transit times error c: {all_obs_tc_c_err}")

### collect all calculated transit times with petigura ephem
all_calc_tc_c = []
for i in range(len(all_transit_num_c)):
    ind = tc_c_petigura + (all_transit_num_c[i] * period_c_petigura)
    all_calc_tc_c.append(ind)
# print(f"All calculated transit times c: {all_calc_tc_c}")

### collect all omc
all_omc_c = []
for i in range(len(all_calc_tc_c)):
    omc_ = all_obs_tc_c[i] - all_calc_tc_c[i]
    all_omc_c.append(omc_)
# print(f"All OMC c: {all_omc_c}")

all_omc_c_err = all_obs_tc_c_err
# print(f"All OMC err c: {all_omc_c_err}")


# Save Results in File

In [51]:
### make txt file with results 
output_file = "ttv_results.txt"

### Combine all data into a structured format
source = ['Narita2015', 'Narita2015', 'Narita2015', 'Narita2015', 'Narita2015', 'Narita2015', 'Narita2015', 'Narita2015', 'Narita2015', 'Narita2015', # b
          'Petigura2020', 'Petigura2020', 'Petigura2020', 'Petigura2020', 'Petigura2020', 'Petigura2020', # b
          'TESS', 'TESS', 'TESS', 'TESS', 'TESS', 'TESS', 'TESS', 'TESS', # b
          'Narita2015', 'Narita2015', 'Narita2015', 'Narita2015', 'Narita2015', 'Narita2015', 'Narita2015', # c
          'Petigura2020', 'Petigura2020'] # c

instrument = ['K2', 'K2', 'K2', 'K2', 'K2', 'K2', 'K2', 'K2', 'K2', 'K2', # b
          'FLWO', 'TRAPPIST', 'MuSCAT', 'Spitzer', 'LCO', 'Spitzer',  # b
          'TESS', 'TESS', 'TESS', 'TESS', 'TESS', 'TESS', 'TESS', 'TESS', # b
          'K2', 'K2', 'K2', 'K2', 'K2', 'K2', 'K2', # c
          'Spitzer', 'Spitzer'] # c

planet_number = []
for i in range(len(all_obs_tc_b)):
    planet_number.append(1)
for i in range(len(all_obs_tc_c)):
    planet_number.append(2)

all_transit_num = [] 
for i in range(len(all_transit_num_b)):
    all_transit_num.append(all_transit_num_b[i])
for i in range(len(all_transit_num_c)):
    all_transit_num.append(all_transit_num_c[i])

all_obs_tc = []
for i in range(len(all_obs_tc_b)):
    all_obs_tc.append(all_obs_tc_b[i])
for i in range(len(all_obs_tc_c)):
    all_obs_tc.append(all_obs_tc_c[i])

all_obs_tc_err = []
for i in range(len(all_obs_tc_b_err)):
    all_obs_tc_err.append(all_obs_tc_b_err[i])
for i in range(len(all_obs_tc_c_err)):
    all_obs_tc_err.append(all_obs_tc_c_err[i])

# all_omc = []
# for i in range(len(all_omc_b)):
#     all_omc.append(all_omc_b[i])
# for i in range(len(all_omc_c)):
#     all_omc.append(all_omc_c[i])

### Ensure source and instrument are numpy arrays with dtype=object
source = np.array(source, dtype=object)
instrument = np.array(instrument, dtype=object)


In [53]:
### save
data_to_save = np.column_stack((planet_number, all_transit_num, all_obs_tc, all_obs_tc_err, source, instrument))

### Define header
header = "Planet_num Index Tc Tc_err Source Instrument"

### Save to file
np.savetxt(output_file, data_to_save, fmt='%d %d %.8f %.8f %s %s', header=header, comments='')

print(f"Results saved to {output_file}")

Results saved to ttv_results.txt
